# 141B Web Technologies for Data Science
# Final Project

### North America Earthquake Analysis

#### Contributor: Chan Woong Joo, Eric Liu, Keerthan Panyala, Jeffery Ugochukwu

-------------------------------------------------------------------------------------------------------------------------------

__Data Extraction using API requests from:__

https://earthquake.usgs.gov/fdsnws/event/1/

In [330]:
import requests
import datetime
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import requests_cache
import reverse_geocoder as rg
requests_cache.install_cache('earthquake_cache')

Getting data from the USGS api using a query method. The params contain the url parameters. 'earthquakes_url' is the 
base url of the usgs api. Used a loop to request data for each year, since each call is limited to 20000 rows. 
Appended the api data to earthquakes_req_df using the for loop. Also, removed uncessary columns

The meanings of the column names are given here: https://earthquake.usgs.gov/data/comcat/data-eventterms.php

In [335]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

earthquakes_params = {'format':'geojson', 'starttime':'', 'endtime':'', 'latitude': '48.368748',
                     'longitude': '-99.996078', 'maxradiuskm': '3500', 'minmagnitude' : '2'}

earthquakes_url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

earthquakes_req_df = pd.DataFrame()

start_end_dates = ['2016-01-01','2017-01-01','2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']

#'maxlatitude': '64.85695', 'maxlongitude': '-68.01198'

for i in range(len(start_end_dates)):
    if earthquakes_params['endtime'] == start_end_dates[-1]:
        break
    else:
        earthquakes_params['starttime'] = start_end_dates[i]
        earthquakes_params['endtime'] = start_end_dates[i+1]
        earthquakes_req = requests.get(earthquakes_url, params = earthquakes_params)
        earthquakes_req_js = earthquakes_req.json()
        data = pd.DataFrame(earthquakes_req_js['features'])
        earthquakes_req_df = earthquakes_req_df.append(data)

earthquakes_req_df = earthquakes_req_df.drop('properties',1).assign(**pd.DataFrame(earthquakes_req_df.properties.values.tolist()))
earthquakes_req_df = earthquakes_req_df.drop('geometry',1).assign(**pd.DataFrame(earthquakes_req_df.geometry.values.tolist()))

earthquakes_req_df = earthquakes_req_df.reset_index(drop=True)

earthquakes_req_df = earthquakes_req_df.drop(columns = ['type','updated', 'tz', 'felt', 'cdi', 'status', 'nst',
                                                        'ids', 'sources', 'types', 'nst', 'magType', 'gap',
                                                        'code','mmi', 'alert', 'net', 'title'])


earthquakes_req_df.head(3)

,id,mag,place,time,url,detail,tsunami,sig,dmin,rms,coordinates
0,ci37387797,2.02,"2km WSW of Brawley, CA",1483228217440,https://earthquake.usgs.gov/earthquakes/eventpage/ci37387797,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37387797&format=geojson,0,63,0.06840,0.23,"[-115.5476667, 32.9691667, 5.63]"
1,ci37775744,2.88,"2km WSW of Brawley, CA",1483228093270,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775744,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775744&format=geojson,0,128,0.06810,0.25,"[-115.5536667, 32.9695, 8.08]"
2,ci37775608,3.63,"1km WSW of Brawley, CA",1483226727120,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775608,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775608&format=geojson,0,204,0.07288,0.29,"[-115.5388333, 32.9748333, 10.2]"


Time was given in milliseconds since the epoch, had to use datetime function to change the time to a readable format. 
Changed the time format to YYYY-MM-DD, HH:MM. Also split the co-ordinated column to makes latitude and longitude.

In [336]:
earthquakes_req_df['time'] = pd.to_datetime(earthquakes_req_df['time'],unit = 'ms').dt.strftime("%Y-%m-%d %H:%M")

earthquakes_req_df['coordinates'] = earthquakes_req_df['coordinates'].astype(str)
earthquakes_req_df['longitude'] = earthquakes_req_df['coordinates'].str[1:] 
earthquakes_req_df['longitude'] = earthquakes_req_df['longitude'].str[:-1]
earthquakes_req_df[['longitude', 'latitude', 'geo_drop']] = earthquakes_req_df['longitude'].str.split(pat = ',', n = 2, expand = True)
earthquakes_req_df = earthquakes_req_df.drop(columns = ['geo_drop', 'coordinates'])

earthquakes_req_df.head(3)



,id,mag,place,time,url,detail,tsunami,sig,dmin,rms,longitude,latitude
0,ci37387797,2.02,"2km WSW of Brawley, CA",2016-12-31 23:50,https://earthquake.usgs.gov/earthquakes/eventpage/ci37387797,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37387797&format=geojson,0,63,0.06840,0.23,-115.5476667,32.9691667
1,ci37775744,2.88,"2km WSW of Brawley, CA",2016-12-31 23:48,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775744,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775744&format=geojson,0,128,0.06810,0.25,-115.5536667,32.9695
2,ci37775608,3.63,"1km WSW of Brawley, CA",2016-12-31 23:25,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775608,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775608&format=geojson,0,204,0.07288,0.29,-115.5388333,32.9748333


In [337]:
#Getting country and state information from the geo-coordinates

earthquakes_req_df['country'] = earthquakes_req_df.apply(lambda x: rg.search((x['latitude'], x['longitude']),mode=1)[0]['cc'], axis = 1)
earthquakes_req_df['state'] = earthquakes_req_df.apply(lambda x: rg.search((x['latitude'], x['longitude']),mode=1)[0]['admin1'], axis = 1)

earthquakes_req_df.to_csv('5yr_north_american_earthquake_data.csv', index = False)

earthquakes_req_df.head()

,id,mag,place,time,url,detail,tsunami,sig,dmin,rms,longitude,latitude,country,state
0,ci37387797,2.02,"2km WSW of Brawley, CA",2016-12-31 23:50,https://earthquake.usgs.gov/earthquakes/eventpage/ci37387797,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37387797&format=geojson,0,63,0.06840,0.23,-115.5476667,32.9691667,US,California
1,ci37775744,2.88,"2km WSW of Brawley, CA",2016-12-31 23:48,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775744,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775744&format=geojson,0,128,0.06810,0.25,-115.5536667,32.9695,US,California
2,ci37775608,3.63,"1km WSW of Brawley, CA",2016-12-31 23:25,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775608,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775608&format=geojson,0,204,0.07288,0.29,-115.5388333,32.9748333,US,California
3,ci37775600,2.20,"2km SW of Brawley, CA",2016-12-31 23:25,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775600,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775600&format=geojson,0,74,0.06603,0.26,-115.544,32.9661667,US,California
4,ci37775536,2.64,"2km W of Brawley, CA",2016-12-31 23:13,https://earthquake.usgs.gov/earthquakes/eventpage/ci37775536,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37775536&format=geojson,0,107,0.06663,0.21,-115.5516667,32.9765,US,California


Creating a local database using sqlalchemy called earthquake_db and saving a dataframe called north_america_earthqukes
to it. Wrote a test query to get number of earthquakes by country. Read in csv in chunks into database.

In [338]:
earthquakes_reader = pd.read_csv('5yr_north_american_earthquake_data.csv', chunksize = 5000, low_memory = False)
earthquakes_chunk = next(earthquakes_reader)

sqlite_file = 'earthquakes.sqlite'
earthquakes_db = sqla.create_engine('sqlite:///' + sqlite_file)

earthquakes_chunk.to_sql('north_america_earthqukes',earthquakes_db, if_exists = 'replace')
for earthquakes_chunk in earthquakes_reader:
    earthquakes_chunk.to_sql('north_america_earthqukes',earthquakes_db, if_exists = 'append')    

pd.read_sql_query("select count(*) as count, country from north_america_earthqukes group by country", earthquakes_db)

,count,country
0,4,BM
1,2804,CA
2,2,CU
3,28,GL
4,1813,MX
5,46364,US


-------------------------------------------------------------------------------------------------------------------------------

__Data Munging__

Pandas

In [339]:
df = earthquakes_req_df

In [340]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51015 entries, 0 to 51014
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         51015 non-null  object 
 1   mag        51015 non-null  float64
 2   place      51015 non-null  object 
 3   time       51015 non-null  object 
 4   url        51015 non-null  object 
 5   detail     51015 non-null  object 
 6   tsunami    51015 non-null  int64  
 7   sig        51015 non-null  int64  
 8   dmin       34432 non-null  float64
 9   rms        51008 non-null  float64
 10  longitude  51015 non-null  object 
 11  latitude   51015 non-null  object 
 12  country    51015 non-null  object 
 13  state      51015 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 5.4+ MB


In [341]:
df['time'] = pd.to_datetime(df['time'])

In [342]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51015 entries, 0 to 51014
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         51015 non-null  object        
 1   mag        51015 non-null  float64       
 2   place      51015 non-null  object        
 3   time       51015 non-null  datetime64[ns]
 4   url        51015 non-null  object        
 5   detail     51015 non-null  object        
 6   tsunami    51015 non-null  int64         
 7   sig        51015 non-null  int64         
 8   dmin       34432 non-null  float64       
 9   rms        51008 non-null  float64       
 10  longitude  51015 non-null  object        
 11  latitude   51015 non-null  object        
 12  country    51015 non-null  object        
 13  state      51015 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(8)
memory usage: 5.4+ MB


In [343]:
df = df.replace(r'^\s*$', np.NaN, regex=True) # Replacing empty string to NaN
df_clean = df[['id','mag','place','time','tsunami','sig','rms','longitude','latitude','country','state']] # Variables of interest

In [344]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51015 entries, 0 to 51014
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         51015 non-null  object        
 1   mag        51015 non-null  float64       
 2   place      51015 non-null  object        
 3   time       51015 non-null  datetime64[ns]
 4   tsunami    51015 non-null  int64         
 5   sig        51015 non-null  int64         
 6   rms        51008 non-null  float64       
 7   longitude  51015 non-null  object        
 8   latitude   51015 non-null  object        
 9   country    51015 non-null  object        
 10  state      51015 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 4.3+ MB


In [345]:
df_clean

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state
0,ci37387797,2.02,"2km WSW of Brawley, CA",2016-12-31 23:50:00,0,63,0.2300,-115.5476667,32.9691667,US,California
1,ci37775744,2.88,"2km WSW of Brawley, CA",2016-12-31 23:48:00,0,128,0.2500,-115.5536667,32.9695,US,California
2,ci37775608,3.63,"1km WSW of Brawley, CA",2016-12-31 23:25:00,0,204,0.2900,-115.5388333,32.9748333,US,California
3,ci37775600,2.20,"2km SW of Brawley, CA",2016-12-31 23:25:00,0,74,0.2600,-115.544,32.9661667,US,California
4,ci37775536,2.64,"2km W of Brawley, CA",2016-12-31 23:13:00,0,107,0.2100,-115.5516667,32.9765,US,California
...,...,...,...,...,...,...,...,...,...,...,...
51010,ok2020aanz,2.50,"14km WSW of Guthrie, Oklahoma",2017-05-18 05:35:00,0,96,0.1600,-97.5553,35.8085,US,Oklahoma
51011,se60272212,2.10,"78km WNW of Skagway, Alaska",2017-05-18 04:54:00,0,68,0.6400,-136.5144,59.7645,US,Alaska
51012,ci39019367,2.10,"29km ENE of Bridgeport, California",2017-05-18 04:50:00,0,68,0.1911,-118.913,38.348,US,California
51013,ok2020aadx,4.40,"153km SSW of Manzanillo, Mexico",2017-05-18 03:20:00,0,298,1.0000,-105.0009,17.8222,MX,Colima


In [346]:
df_dna_clean = df_clean.dropna(axis=0) # Dropping rows with NaN
df_dna_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51008 entries, 0 to 51014
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         51008 non-null  object        
 1   mag        51008 non-null  float64       
 2   place      51008 non-null  object        
 3   time       51008 non-null  datetime64[ns]
 4   tsunami    51008 non-null  int64         
 5   sig        51008 non-null  int64         
 6   rms        51008 non-null  float64       
 7   longitude  51008 non-null  object        
 8   latitude   51008 non-null  object        
 9   country    51008 non-null  object        
 10  state      51008 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 4.7+ MB


Reassigning values for 'tsunami' variable

In [347]:
pd.options.mode.chained_assignment = None  # default='warn'
df_dna_clean['tsunami'] = df_dna_clean.tsunami.replace(to_replace =[0,1], value=['No','Yes']) # This gave warning for not using df.loc
#df_dna_clean.reset_index() # Reseting index once
df_dna_clean

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state
0,ci37387797,2.02,"2km WSW of Brawley, CA",2016-12-31 23:50:00,No,63,0.2300,-115.5476667,32.9691667,US,California
1,ci37775744,2.88,"2km WSW of Brawley, CA",2016-12-31 23:48:00,No,128,0.2500,-115.5536667,32.9695,US,California
2,ci37775608,3.63,"1km WSW of Brawley, CA",2016-12-31 23:25:00,No,204,0.2900,-115.5388333,32.9748333,US,California
3,ci37775600,2.20,"2km SW of Brawley, CA",2016-12-31 23:25:00,No,74,0.2600,-115.544,32.9661667,US,California
4,ci37775536,2.64,"2km W of Brawley, CA",2016-12-31 23:13:00,No,107,0.2100,-115.5516667,32.9765,US,California
...,...,...,...,...,...,...,...,...,...,...,...
51010,ok2020aanz,2.50,"14km WSW of Guthrie, Oklahoma",2017-05-18 05:35:00,No,96,0.1600,-97.5553,35.8085,US,Oklahoma
51011,se60272212,2.10,"78km WNW of Skagway, Alaska",2017-05-18 04:54:00,No,68,0.6400,-136.5144,59.7645,US,Alaska
51012,ci39019367,2.10,"29km ENE of Bridgeport, California",2017-05-18 04:50:00,No,68,0.1911,-118.913,38.348,US,California
51013,ok2020aadx,4.40,"153km SSW of Manzanillo, Mexico",2017-05-18 03:20:00,No,298,1.0000,-105.0009,17.8222,MX,Colima


In [348]:
year = df_dna_clean['time'].dt.year
month = df_dna_clean['time'].dt.month
df_dna_clean['count_month'] = df_dna_clean.groupby([year,month])['time'].transform('size')
df_dna_clean['count_day'] = df_dna_clean.groupby('time')['time'].transform('size')

In [349]:
df_dna_clean

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state,count_month,count_day
0,ci37387797,2.02,"2km WSW of Brawley, CA",2016-12-31 23:50:00,No,63,0.2300,-115.5476667,32.9691667,US,California,3360,5
1,ci37775744,2.88,"2km WSW of Brawley, CA",2016-12-31 23:48:00,No,128,0.2500,-115.5536667,32.9695,US,California,3360,5
2,ci37775608,3.63,"1km WSW of Brawley, CA",2016-12-31 23:25:00,No,204,0.2900,-115.5388333,32.9748333,US,California,3360,10
3,ci37775600,2.20,"2km SW of Brawley, CA",2016-12-31 23:25:00,No,74,0.2600,-115.544,32.9661667,US,California,3360,10
4,ci37775536,2.64,"2km W of Brawley, CA",2016-12-31 23:13:00,No,107,0.2100,-115.5516667,32.9765,US,California,3360,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51010,ok2020aanz,2.50,"14km WSW of Guthrie, Oklahoma",2017-05-18 05:35:00,No,96,0.1600,-97.5553,35.8085,US,Oklahoma,642,2
51011,se60272212,2.10,"78km WNW of Skagway, Alaska",2017-05-18 04:54:00,No,68,0.6400,-136.5144,59.7645,US,Alaska,642,2
51012,ci39019367,2.10,"29km ENE of Bridgeport, California",2017-05-18 04:50:00,No,68,0.1911,-118.913,38.348,US,California,642,2
51013,ok2020aadx,4.40,"153km SSW of Manzanillo, Mexico",2017-05-18 03:20:00,No,298,1.0000,-105.0009,17.8222,MX,Colima,642,4


Counting number of Tsunamis

In [350]:
df_dna_clean['tsunami'].isin(['Yes']).sum(axis=0)

194

__Possible Question:__
What are the top 10 states with the most number of earthquakes?

In [351]:
n = 10
df_dna_clean['state'].value_counts()[:n].index.tolist()

['California',
 'Oklahoma',
 'Alaska',
 'Nevada',
 'Idaho',
 'Montana',
 'Kansas',
 'Oregon',
 'Wyoming',
 'Yukon']

__Possible Question:__

In [352]:
max = df_dna_clean['mag'].groupby(df.time.dt.month).max()
max

time
1     6.6
2     5.1
3     5.5
4     4.7
5     5.7
6     6.3
7     5.8
8     5.1
9     7.1
10    5.6
11    5.7
12    6.6
Name: mag, dtype: float64

__Possible Question:__
From the data, how many earthquakes had magnitude more than 6.0?

In [353]:
mag_6_above = df_dna_clean.loc[df_dna_clean['mag'] >= 6.0]
mag_6_above

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state,count_month,count_day
551,us20007z6r,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
4168,us200062i1,6.3,"106km SSW of San Patricio, Mexico",2016-06-07 10:51:00,No,611,1.00,-105.1731,18.3637,MX,Jalisco,2785,5
7099,us10004g4l,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
8190,ci37821959,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
11807,mb80234729,6.3,"106km SSW of San Patricio, Mexico",2016-06-07 10:51:00,No,611,1.00,-105.1731,18.3637,MX,Jalisco,2785,5
14738,ismpkansas70222558,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
16464,ci38148879,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
20081,ak018adsqli3,6.3,"106km SSW of San Patricio, Mexico",2016-06-07 10:51:00,No,611,1.00,-105.1731,18.3637,MX,Jalisco,2785,5
23012,nc72989841,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
25102,ak019fj4fgey,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5


__Possible Question:__ How many earthquakes caused tsunami?

In [354]:
tsunami = df_dna_clean.loc[(df_dna_clean['tsunami'] == 'Yes')]
tsunami

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state,count_month,count_day
252,ak016glf20b9,4.10,"71km E of Cape Yakataga, Alaska",2016-12-26 23:59:00,Yes,259,0.83,-141.1619,60.1571,US,Alaska,3360,5
448,nc72737985,5.01,"8km NW of The Geysers, California",2016-12-14 16:41:00,Yes,976,0.06,-122.8413333,38.8221667,US,California,3360,5
452,nc72737765,3.95,"9km W of Templeton, California",2016-12-14 11:58:00,Yes,670,0.07,-120.805,35.5606667,US,California,3360,5
545,us20007z7b,4.70,"179km W of Ferndale, California",2016-12-08 16:32:00,Yes,362,0.99,-126.3674,40.4292,US,California,3360,5
551,us20007z6r,6.60,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49876,nc73349005,4.30,"83km ESE of Whittier, Alaska",2017-07-11 09:30:00,Yes,287,0.80,-147.3576,60.4302,US,Alaska,1454,2
50242,us70007q8c,4.03,"28km SW of Rio Dell, California",2017-06-24 21:22:00,Yes,459,0.12,-124.2988333,40.2878333,US,California,1222,2
50276,us70007lee,4.10,"31km WNW of Valdez, Alaska",2017-06-22 15:21:00,Yes,267,0.74,-146.8244,61.2549,US,Alaska,1222,2
50328,ok2020couz,3.90,"88km NNW of Tofino, Canada",2017-06-19 17:22:00,Yes,244,1.19,-126.5009,49.8501,CA,British Columbia,1222,2


__Possible Question:__
How many earthquakes had magnitude over 6.0 AND caused tsunami?

In [355]:
over_6_tsunami = df_dna_clean.loc[(df_dna_clean['tsunami'] == 'Yes') & (df_dna_clean['mag'] >= 6.0)]
over_6_tsunami

,id,mag,place,time,tsunami,sig,rms,longitude,latitude,country,state,count_month,count_day
551,us20007z6r,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
7099,us10004g4l,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
8190,ci37821959,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
14738,ismpkansas70222558,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
16464,ci38148879,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
23012,nc72989841,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
25102,ak019fj4fgey,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
31650,ci38469839,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
38380,us7000cq6p,6.6,"164km W of Ferndale, California",2016-12-08 14:49:00,Yes,1120,1.11,-126.1936,40.4535,US,California,3360,5
44928,mb80431404,6.6,"215km SW of Tomatlan, Mexico",2016-01-21 18:06:00,Yes,673,0.98,-106.9337,18.8239,MX,Jalisco,3855,5
